In [89]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
import torch
import sys
import pandas as pd
import numpy as np
import os
import random
import json

In [88]:
!git clone https://github.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words
!git clone https://github.com/doublems/korean-bad-words
!git clone https://github.com/organization/Gentleman/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words'...
remote: Enumerating objects: 480, done.
remote: Total 480 (delta 0), reused 0 (delta 0), pack-reused 480
Receiving objects: 100% (480/480), 110.14 KiB | 5.01 MiB/s, done.
Resolving deltas: 100% (249/249), done.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'korean-bad-words'...
remote: Enumerating objects: 56, d

In [90]:
# 비속어 사전 01
slang_list_01 = []
f = open("./List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/ko", 'r')
lines = f.readlines()
for line in lines:
    line = line.replace("\n", "")
    slang_list_01.append(line)
f.close()

# 비속어 사전 02
slang_list_02 = []
f = open("./korean-bad-words/korean-bad-words.md", 'r')
lines = f.readlines()
for line in lines:
    line = line.replace("\n", "")
    slang_list_02.append(line)
f.close()

# 비속어 사전 03
with open('./Gentleman/resources/badwords.json') as json_file:
    json_data = json.load(json_file)
slang_list_03 = json_data["badwords"]

# 비속어 사전 통합
slang_list = slang_list_01 + slang_list_02 + slang_list_03
slang_list = list(set(slang_list))

In [98]:
def find_Badword(sent) :
    Badin = False
    for word in slang_list :
        if word in sent :
            Badin = True
    return Badin

In [2]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
seed_everything()

In [5]:
dataset = pd.read_csv('ChatbotData.csv')

In [6]:
dataset['label'].unique()

array([0, 1, 2])

In [7]:
dataset.drop_duplicates(subset=['Q'], inplace=True)
dataset['label'][8860]

2

In [8]:
# dataset_random = dataset.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset
# dataset_random
dataset_zero=pd.DataFrame({"Question":dataset['Q'][:5261], "answer":dataset['A'][:5261]})
dataset_one=pd.DataFrame({"Question":dataset['Q'][5261:8743], "answer":dataset['A'][5261:8743]})
dataset_two=pd.DataFrame({"Question":dataset['Q'][8743:], "answer":dataset['A'][8743:]})

len(dataset_two)

2919

In [9]:
dataset_random = dataset.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset
dataset_random
dataset = dataset_random

In [10]:
dataset

,Q,A,label
0,사랑하는 사람 잊는 법,충분히 아파하면서 충분한 시간이 필요할 거예요.,2
1,술 안 먹으면 친구랑 뭐하지,아무것도 안해도 재밌을 거예요.,0
2,짝남이 고시생이면 기다려야 하나요?,기다릴 수 있다면 기다리세요.,2
3,이젠 짝사랑하는게 무섭고 두려워.,이제 짝사랑 말고 둘이서 하는 사랑하세요.,2
4,숨막히네,숨 쉴 수 있을 때가 올 거예요.,1
...,...,...,...
11657,짝남한테 같이 공부하러 가자고 해볼까?,공부 도와달라고 해보세요.,2
11658,회사 일이 너무 많아,너무 무리하지 마세요.,0
11659,9년이 넘는 연애를 끝냅니다,공허함이 더 크시겠어요.,1
11660,내가 좋아하는 가수 컴백한다,기다렸나봐요.,0


In [11]:
train_data_Qusetion = pd.DataFrame({"document" : dataset['Q'][:7000], "label" : dataset['label'][:7000]})
train_data_Answer = pd.DataFrame({"document" : dataset['A'][:7000], "label" : dataset['label'][:7000]})
valid_data_Qusetion = pd.DataFrame({"document" : dataset['Q'][7000:9000], "label" : dataset['label'][7000:9000]})
valid_data_Answer = pd.DataFrame({"document" : dataset['A'][7000:9000], "label" : dataset['label'][7000:9000]})
test_data_Qusetion = pd.DataFrame({"document" : dataset['Q'][9000:], "label" : dataset['label'][9000:]})
test_data_Answer = pd.DataFrame({"document" : dataset['A'][9000:], "label" : dataset['label'][9000:]})
train_data_Qusetion['document'].replace('', np.nan,inplace=True)
valid_data_Qusetion['document'].replace('', np.nan,inplace=True)
test_data_Qusetion['document'].replace('', np.nan,inplace=True)

In [12]:
train_data_Qusetion['label'].value_counts()

0    3184
1    2084
2    1732
Name: label, dtype: int64

In [13]:
valid_data_Qusetion['label'].value_counts()

0    905
1    581
2    514
Name: label, dtype: int64

In [14]:
test_data_Answer['label'].value_counts()

0    1172
1     817
2     673
Name: label, dtype: int64

In [15]:
train_data_Qusetion = pd.DataFrame({"document" : dataset['Q'][:7000], "label" : dataset['label'][:7000]})
train_data_Answer = pd.DataFrame({"document" : dataset['A'][:7000], "label" : dataset['label'][:7000]})
valid_data_Qusetion = pd.DataFrame({"document" : dataset['Q'][7000:9000], "label" : dataset['label'][7000:9000]})
valid_data_Answer = pd.DataFrame({"document" : dataset['A'][7000:9000], "label" : dataset['label'][7000:9000]})
test_data_Qusetion = pd.DataFrame({"document" : dataset['Q'][9000:], "label" : dataset['label'][9000:]})
test_data_Answer = pd.DataFrame({"document" : dataset['A'][9000:], "label" : dataset['label'][9000:]})

In [16]:
train_data_Qusetion['document'].replace('', np.nan,inplace=True)
valid_data_Qusetion['document'].replace('', np.nan,inplace=True)
test_data_Qusetion['document'].replace('', np.nan,inplace=True)
# train_data_Answer['document'].replace('', np.nan,inplace=True)
# test_data_Answer['document'].replace('', np.nan,inplace=True)
# train_data_Qusetion = train_data_Qusetion.dropna(how = 'any')
# test_data_Qusetion = test_data_Qusetion.dropna(how = 'any')
# train_data_Qusetion = train_data_Answer.dropna(how = 'any')
# test_data_Answer = test_data_Answer.dropna(how = 'any')

In [17]:
print(len(train_data_Answer))
print(len(test_data_Qusetion))
print(len(train_data_Answer))
print(len(test_data_Answer))
print(len(valid_data_Answer))
print(len(valid_data_Qusetion))

7000
2662
7000
2662
2000
2000


In [18]:
MD_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MD_NAME)

In [19]:
tokenizer_train_sentences = tokenizer(
    list(train_data_Qusetion['document']),
    return_tensors="pt",
    padding = True,
    truncation=True,
    add_special_tokens=True
)

In [20]:
tokenizer_valid_sentences = tokenizer(
    list(valid_data_Qusetion['document']),
    return_tensors="pt",
    padding = True,
    truncation=True,
    add_special_tokens=True
)

In [21]:
tokenizer_test_sentences = tokenizer(
    list(test_data_Qusetion['document']),
    return_tensors="pt",
    padding = True,
    truncation=True,
    add_special_tokens=True
)

In [22]:
train_label = train_data_Qusetion['label'].values
test_label = test_data_Qusetion['label'].values
valid_label = valid_data_Qusetion['label'].values

In [23]:
list(set(valid_label))

[0, 1, 2]

In [24]:
len(valid_label)

2000

In [25]:
print(tokenizer_train_sentences[0])
print(tokenizer_train_sentences[0].tokens)
print(tokenizer_train_sentences[0].ids)
print(tokenizer_train_sentences[0].attention_mask)

Encoding(num_tokens=36, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '사', '##랑', '##하는', '사', '##람', '잊', '##는', '법', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 9405, 62200, 12178, 9405, 61250, 9649, 11018, 9341, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
print(tokenizer_test_sentences[0])
print(tokenizer_test_sentences[0].tokens)
print(tokenizer_test_sentences[0].ids)
print(tokenizer_test_sentences[0].attention_mask)

Encoding(num_tokens=34, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '나', '##한', '##테', '##만', '##은', '완', '##전', '솔', '##직', '##했', '##으면', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 8982, 11102, 119351, 19105, 10892, 9591, 16617, 9451, 33077, 119424, 80046, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [27]:
from torch.utils.data import Dataset

In [28]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader

In [29]:
class SingleSentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [30]:
tokenizer_train_sentences['input_ids']

tensor([[   101,   9405,  62200,  ...,      0,      0,      0],
        [   101,   9463,   9521,  ...,      0,      0,      0],
        [   101,   9716,  37004,  ...,      0,      0,      0],
        ...,
        [   101,   8987, 119196,  ...,      0,      0,      0],
        [   101,   9568, 119121,  ...,      0,      0,      0],
        [   101,   9781,  17196,  ...,      0,      0,      0]])

In [31]:
#dataset 생성
train_dataset = SingleSentDataset(tokenizer_train_sentences, train_label)
valid_dataset = SingleSentDataset(tokenizer_valid_sentences, valid_label)
test_dataset = SingleSentDataset(tokenizer_test_sentences, test_label)

In [32]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [33]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='logs',            # directory for storing logs
    logging_steps=10,
    save_steps=300,
    save_total_limit=2,
    metric_for_best_model="accuracy"
)

In [34]:
from transformers import BertForSequenceClassification, BertConfig

config = BertConfig.from_pretrained(MD_NAME)
config.num_labels = 3
model = BertForSequenceClassification(config) 
model.parameters

<bound method Module.parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [35]:
# from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
#     acc = accuracy_score(labels, preds)
#     return acc

In [36]:
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [37]:
# model = BertForSequenceClassification.from_pretrained(MD_NAME)
# model.classifier.out_features = 3
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # training dataset
    #vocab_size=500000
    compute_metrics = compute_metrics
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [38]:
# import tensorboard
# %load_ext tensorboard
# %tensorboard --logdir logs/

In [39]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7000
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4380
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: ddrongy (winner). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/tmp/ipykernel_11898/3231692829.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,0.703000,0.586598,0.758000
2,0.488000,0.618328,0.767000
3,0.440400,0.572009,0.790000
4,0.455600,0.507587,0.818500
5,0.372600,0.561121,0.811500
6,0.300500,0.610135,0.813500
7,0.255500,0.709696,0.808000
8,0.172100,0.814616,0.813500
9,0.112900,0.920227,0.811000
10,0.130400,0.939265,0.814500


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-438
Configuration saved in ./results/checkpoint-438/config.json
Model weights saved in ./results/checkpoint-438/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-3942] due to args.save_total_limit
/tmp/ipykernel_11898/3231692829.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-876
Configuration saved in ./results/checkpoint-876/config.json
Model weights saved in ./results/checkpoint-876/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-4380] due to args.save_total_limit
/tmp/ipykernel_11898/3231692

TrainOutput(global_step=4380, training_loss=0.37397694903081413, metrics={'train_runtime': 998.4711, 'train_samples_per_second': 70.107, 'train_steps_per_second': 4.387, 'total_flos': 1295011352880000.0, 'train_loss': 0.37397694903081413, 'epoch': 10.0})

In [40]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics
)

In [42]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 2662
  Batch size = 32
/tmp/ipykernel_11898/3231692829.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


{'eval_loss': 0.9356488585472107,
 'eval_accuracy': 0.8110443275732532,
 'eval_f1': 0.80923975594823,
 'eval_precision': 0.8101369347176312,
 'eval_recall': 0.8084504381873856,
 'eval_runtime': 6.4986,
 'eval_samples_per_second': 409.625,
 'eval_steps_per_second': 12.926}

In [87]:
# predict함수
def sentences_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=128
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

In [44]:
print(sentences_predict("영화 개재밌어 ㅋㅋㅋㅋㅋ"))
print(sentences_predict("진짜 재미없네요 ㅋㅋ"))
print(sentences_predict("너 때문에 진짜 짜증나"))
print(sentences_predict("정말 재밌고 좋았어요."))

2
1
1
2


In [45]:
from transformers import pipeline

nlp_sentence_classif = pipeline('sentiment-analysis',model=model, tokenizer=tokenizer, device=0)

##test
print(nlp_sentence_classif('영화 개재밌어 ㅋㅋㅋㅋㅋ'))
print(nlp_sentence_classif('진짜 재미없네요 ㅋㅋ'))
print(nlp_sentence_classif('너 때문에 진짜 짜증나'))
print(nlp_sentence_classif('정말 재밌고 좋았어요.'))

[{'label': 'LABEL_2', 'score': 0.9979351758956909}]
[{'label': 'LABEL_1', 'score': 0.9485467672348022}]
[{'label': 'LABEL_1', 'score': 0.9443480968475342}]
[{'label': 'LABEL_2', 'score': 0.998594343662262}]


In [49]:
print(nlp_sentence_classif('김규리 바보'))

[{'label': 'LABEL_0', 'score': 0.9967002272605896}]


In [47]:
#cls 유사도 구하는 부분
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer_cls = AutoTokenizer.from_pretrained(MODEL_NAME)
model_cls = AutoModel.from_pretrained(MODEL_NAME)

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /opt/ml/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "a

In [51]:
chatbot_zero_Qlist = []
for data in dataset_zero['Question']:
    chatbot_zero_Qlist.append(data)
    
chatbot_one_Qlist = []
for data in dataset_one['Question']:
    chatbot_one_Qlist.append(data)
    
chatbot_two_Qlist = []
for data in dataset_two['Question']:
    chatbot_two_Qlist.append(data)

In [55]:
chatbot_zero_Alist = []
for data in dataset_zero['answer']:
    chatbot_zero_Alist.append(data)

chatbot_one_Alist = []
for data in dataset_one['answer']:
    chatbot_one_Alist.append(data)
    
chatbot_two_Alist = []
for data in dataset_two['answer']:
    chatbot_two_Alist.append(data)

In [59]:
def get_cls_token(sent) : 
  model_cls.eval()
  tokenized_sent = tokenizer_cls(
      sent,
      return_tensors = "pt",
      truncation = True,
      add_special_tokens=True,
      max_length = 128
  )
  with torch.no_grad():
    outputs = model_cls(**tokenized_sent)
  logits = outputs.last_hidden_state[:,0,:].detach().cpu().numpy()
  return logits

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

In [61]:
data_cls_zero = []
data_cls_one = []
data_cls_two = []

for q in tqdm(chatbot_zero_Qlist) :
  q_cls = get_cls_token(q)
  data_cls_zero.append(q_cls)
data_cls_zero = np.array(data_cls_zero).squeeze(axis=1)

for q in tqdm(chatbot_one_Qlist) :
  q_cls = get_cls_token(q)
  data_cls_one.append(q_cls)
data_cls_one = np.array(data_cls_one).squeeze(axis=1)

for q in tqdm(chatbot_two_Qlist) :
  q_cls = get_cls_token(q)
  data_cls_two.append(q_cls)
data_cls_two = np.array(data_cls_two).squeeze(axis=1)

100%|██████████| 2919/2919 [08:57<00:00,  5.43it/s]


In [106]:
query = '라디라펌'
query_cls_hidden = get_cls_token(query)
print(sentences_predict(query))

0


In [107]:
if find_Badword(query) :
    print('욕설이 포함된 질문입니다')

elif sentences_predict(query) == 0:
    cos_sim = cosine_similarity(query_cls_hidden, data_cls_zero)

    top_question = np.argmax(cos_sim)

    print('나의 질문: ', query)
    print('저장된 답변: ', chatbot_zero_Alist[top_question])

elif sentences_predict(query) == 1:
    cos_sim = cosine_similarity(query_cls_hidden, data_cls_one)

    top_question = np.argmax(cos_sim)

    print('나의 질문: ', query)
    print('저장된 답변: ', chatbot_one_Alist[top_question])
    
elif sentences_predict(query) == 2:
    cos_sim = cosine_similarity(query_cls_hidden, data_cls_two)

    top_question = np.argmax(cos_sim)
    
    print('나의 질문: ', query)
    print('저장된 답변: ', chatbot_two_Alist[top_question])


나의 질문:  라디라펌
저장된 답변:  있으면 좋죠.
